## Odometer_Readings_Two_Test_Dates

### make two different columns for the vehicle like two odometer readings for two different test dates

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Extracting ZIP Code Prefixes & Matching with VINs.csv")
df.head()

,vin,zipcode,odometer,test_date,make,model,my,purchase_date
0,WBAEV33452KL68783,60655,125000,6/1/2020,BMW,3 Series,2002,5/1/2021
1,1GBFG15R6Y1100200,60632,206000,6/1/2020,CHEV,Express Cargo,2000,5/21/2017
2,1GCDT136548185796,60501,109000,6/1/2020,CHEV,Colorado,2004,6/18/2013
3,2GKALMEK0C6364495,60453,80000,6/1/2020,GMC,Terrain,2012,4/26/2018
4,1HGCG1652YA094701,60459,167000,6/1/2020,HOND,Accord,2000,7/19/2014


In [3]:
df.shape

(8338418, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8338418 entries, 0 to 8338417
Data columns (total 8 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   vin            object
 1   zipcode        int64 
 2   odometer       int64 
 3   test_date      object
 4   make           object
 5   model          object
 6   my             int64 
 7   purchase_date  object
dtypes: int64(3), object(5)
memory usage: 508.9+ MB


In [5]:
df.isnull().sum()

vin              0
zipcode          0
odometer         0
test_date        0
make             0
model            0
my               0
purchase_date    0
dtype: int64

In [6]:
# Find duplicate rows
duplicates = df[df.duplicated()]
print("Total duplicate rows:", len(duplicates))

# Display duplicate rows
duplicates.head()


Total duplicate rows: 37406


,vin,zipcode,odometer,test_date,make,model,my,purchase_date
423,2G4WS52J351137092,60433,0,6/1/2020,BUIC,Century,2005,12/1/2005
463,5Y2SL65816Z435471,60543,0,6/1/2020,PONT,Vibe,2006,7/14/2018
581,JTKKUPB46E1042219,60608,23000,6/1/2020,TOYT,xD,2014,4/9/2014
2487,1G1PF5SCXC7346028,60185,132000,6/1/2020,CHEV,Cruze,2012,3/14/2018
2716,1G4PP5SKXE4182274,60403,19000,6/1/2020,BUIC,Verano,2014,5/9/2016


In [7]:
# Display the first few duplicate rows
print(duplicates.head())

# Check if all columns are identical for duplicates
print("Are all columns identical in duplicates?")
print(df[df.duplicated()].equals(df[df.duplicated(keep=False)]))


                    vin  zipcode  odometer test_date  make    model    my  \
423   2G4WS52J351137092    60433         0  6/1/2020  BUIC  Century  2005   
463   5Y2SL65816Z435471    60543         0  6/1/2020  PONT     Vibe  2006   
581   JTKKUPB46E1042219    60608     23000  6/1/2020  TOYT       xD  2014   
2487  1G1PF5SCXC7346028    60185    132000  6/1/2020  CHEV    Cruze  2012   
2716  1G4PP5SKXE4182274    60403     19000  6/1/2020  BUIC   Verano  2014   

     purchase_date  
423      12/1/2005  
463      7/14/2018  
581       4/9/2014  
2487     3/14/2018  
2716      5/9/2016  
Are all columns identical in duplicates?
False


The analysis first focused on identifying duplicate records within the dataset. By inspecting duplicates, it was determined that not all columns were identical in the duplicate entries. This suggests that some records might have inconsistencies or variations in specific attributes such as odometer readings or ZIP codes.

In [8]:
# Check for duplicates based on 'vin' and 'test_date'
partial_duplicates = df[df.duplicated(subset=['vin', 'test_date'], keep=False)]
print("Partial duplicates based on 'vin' and 'test_date':")
print(partial_duplicates)


Partial duplicates based on 'vin' and 'test_date':
                       vin  zipcode  odometer  test_date  make        model  \
37       5J6RE4H34AL051901    60629     60000   6/1/2020  HOND         CR-V   
62       4T1BG22KXYU650856    60164      1000   6/1/2020  TOYT        Camry   
97       JTKKUPB46E1042219    60608     23000   6/1/2020  TOYT           xD   
407      2G4WS52J351137092    60433         0   6/1/2020  BUIC      Century   
408      5TDZA23CX6S505619    60645    265000   6/1/2020  TOYT       Sienna   
...                    ...      ...       ...        ...   ...          ...   
8338329  1NXBU40EX9Z030480    60626    189000  8/31/2024  TOYT      Corolla   
8338333  1G1PH5S97B7135866    60643     54000  8/31/2024  CHEV        Cruze   
8338357  1GNEK13T61J301869    60123    201000  8/31/2024  CHEV        Tahoe   
8338369  2GNALDEK9D6402195    60608    118000  8/31/2024  CHEV      Equinox   
8338410  3GTEK13358G235173    60099    211000  8/31/2024  GMC   Sierra 1500   



To refine the duplicate analysis, the dataset was checked for partial duplicates based on `vin` (Vehicle Identification Number) and `test_date`. This step revealed that a significant number of records had the same VIN and test date but potentially differed in other attributes. This highlights data redundancy or potential errors in the dataset.


In [9]:
# Aggregate duplicate rows by grouping by 'vin' and 'test_date'
df = df.groupby(['vin', 'test_date'], as_index=False).agg({
    'zipcode': 'first',       
    'odometer': 'mean',       
    'make': 'first',         
    'model': 'first',         
    'my': 'first',            
    'purchase_date': 'min'    
})

# Display results after aggregation
print(df.head())


                   vin  test_date  zipcode  odometer  make   model    my  \
0  10000000000003412WI   9/8/2020    60505  114000.0  CHEV          1999   
1  10000000000005614WI  8/26/2024    60174  115000.0  FORD          2005   
2  10000000000005641WI  11/5/2021    60041   99000.0  FORD          2005   
3    107HA18N92J225350  1/16/2020    60804  216000.0  DODG          2002   
4     11ND52J43M623642  8/10/2021    60617  156000.0  CHEV  MAL     2003   

  purchase_date  
0      3/4/2011  
1     9/15/2023  
2     10/2/2018  
3    10/15/2018  
4      1/1/1900  


To resolve duplicate entries, an aggregation method was applied. The data was grouped by `vin` and `test_date`, ensuring that for each unique combination, key attributes were retained in a structured manner:
- **Zipcode**: The first occurrence was kept.
- **Odometer Reading**: The average value was taken.
- **Make, Model, and Model Year**: The first occurrence was retained.
- **Purchase Date**: The earliest recorded purchase date was selected.

This aggregation approach ensures a more consistent and reliable dataset while removing redundancy, ultimately improving data quality for further analysis.

In [10]:
# Check for duplicates
remaining_duplicates = df[df.duplicated(subset=['vin', 'test_date'])]
print(f"Remaining duplicate rows: {len(remaining_duplicates)}")


Remaining duplicate rows: 0


In [11]:
# Check descriptive statistics for numerical columns
print(df['odometer'].describe())

# Check unique values for categorical columns
print(df['make'].unique())


count    8.207797e+06
mean     1.012595e+05
std      6.113666e+04
min      0.000000e+00
25%      5.600000e+04
50%      9.200000e+04
75%      1.390000e+05
max      9.990000e+05
Name: odometer, dtype: float64
['CHEV' 'FORD' 'DODG' 'ACUR' 'ALFA' 'HOND' 'AC  ' 'HYUN' 'CHRY' 'CHRS'
 'DESO' 'Chry' 'JEEP' 'JENS' 'Jeep' 'Dodg' 'RAM ' 'IM92' 'Ram' 'DAIM'
 'RAM' 'SUBA' 'Ford' 'TRAP' 'HZ58' 'GX11' 'THMH' 'AXS1' 'UTIM' 'GW81'
 'ID66' 'READ' 'HV11' 'MIBR' 'GA77' 'MBMB' 'IR13' 'STAO' 'GU01' 'ELKC'
 'SRCR' 'ELKD' 'STR1' 'DMND' 'CHAM' 'GEN ' 'STTT' 'IP44' 'WORL' 'SENA'
 'GOSH' 'IQ29' 'SCSL' 'COAM' 'CRFT' 'COAH' 'FOUW' 'MNAC' 'RKPT' 'THOR'
 'SCSQ' 'WINN' 'MJST' 'MIWI' 'FD83' 'GB65' 'CHAT' 'IM73' 'MTMH' 'FIAT'
 'GMC ' 'TOYT' 'ENGF' 'BMW ' 'NISS' 'FRHT' 'Chev' 'HP10' 'CADI' 'PONT'
 'BUIC' 'Pont' 'OLDS' 'HF49' 'MITS' 'STRN' 'SATU' 'ICRP' 'ZZ19' 'STPR'
 'FRDE' 'BYBR' 'TTBS' 'FZ76' 'GLF ' 'GZ05' 'IF75' 'IC61' 'ISU ' 'GMC'
 'INTL' 'HV03' 'GEO ' 'CHBS' 'HIND' 'AMER' 'LEXS' 'LINC' 'MERC' 'MERB'
 'DATS' 'TOYO' 

In [12]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)


Missing values in each column:
vin              0
test_date        0
zipcode          0
odometer         0
make             0
model            0
my               0
purchase_date    0
dtype: int64


In [13]:
# Ensure 'test_date' is in datetime format

df['test_date'] = pd.to_datetime(df['test_date'])

# Extract unique years
unique_years = df['test_date'].dt.year.unique()
print(f"Unique years in the dataset: {unique_years}")


Unique years in the dataset: [2020 2024 2021 2023]


To analyze trends over time, the `test_date` column was converted to datetime format, and the unique years were extracted. The dataset contained records from the years **2020, 2021, 2023, and 2024**. This information is useful for time-based analysis, such as identifying trends in vehicle inspections over different years.

In [14]:
df['test_date'].unique()

array(['2020-09-08T00:00:00.000000000', '2024-08-26T00:00:00.000000000',
       '2021-11-05T00:00:00.000000000', ...,
       '2021-01-18T00:00:00.000000000', '2024-05-05T00:00:00.000000000',
       '2020-11-27T00:00:00.000000000'], dtype='datetime64[ns]')

In [15]:
# Convert 'test_date' to datetime format for consistency
df['test_date'] = pd.to_datetime(df['test_date'])

# Define two specific test dates
date1 = '2020-09-08T00:00:00.000000000'
date2 = '2021-11-05T00:00:00.000000000'

# Filter data for these two test dates
df_date1 = df[df['test_date'] == date1][['vin', 'odometer']].rename(columns={'odometer': 'odometer_date1'})
df_date2 = df[df['test_date'] == date2][['vin', 'odometer']].rename(columns={'odometer': 'odometer_date2'})


To analyze vehicle odometer readings over time, the dataset was filtered for two specific test dates: **September 8, 2020, and November 5, 2021**. The odometer readings for these dates were extracted for each **VIN (Vehicle Identification Number)**, creating two separate datasets: 
- `df_date1` containing VINs and their odometer readings for the first test date.
- `df_date2` containing VINs and their odometer readings for the second test date.

In [16]:
# Merge the two DataFrames on 'vin'
df_combined = pd.merge(df_date1, df_date2, on='vin', how='inner')

# Display the resulting DataFrame
print(df_combined.head())


Empty DataFrame
Columns: [vin, odometer_date1, odometer_date2]
Index: []


The two datasets were merged based on `vin` to compare odometer readings across these dates. However, the merged DataFrame resulted in an **empty DataFrame**, indicating that **no common VINs existed between the two test dates**.

In [17]:
# Check the number of rows in df_date1 and df_date2
print(f"Rows in df_date1 (date1): {len(df_date1)}")
print(f"Rows in df_date2 (date2): {len(df_date2)}")


Rows in df_date1 (date1): 9636
Rows in df_date2 (date2): 6915


In [18]:
# Check for common VINs between df_date1 and df_date2
common_vins = set(df_date1['vin']).intersection(set(df_date2['vin']))
print(f"Number of common VINs: {len(common_vins)}")


Number of common VINs: 0


A check was performed to find common VINs between the two test dates. The result showed **zero common VINs**, meaning that no vehicle appeared in both test dates in this dataset. This could indicate that vehicles were not tested repeatedly or that there are inconsistencies in the dataset that need further exploration.

These findings suggest that further data validation and additional filtering criteria might be necessary to track vehicles over multiple test dates.


In [19]:
# Standardize 'vin' column by stripping spaces and converting to uppercase
df['vin'] = df['vin'].str.strip().str.upper()

# Re-create df_date1 and df_date2 after standardizing
df_date1 = df[df['test_date'] == date1][['vin', 'odometer']].rename(columns={'odometer': 'odometer_date1'})
df_date2 = df[df['test_date'] == date2][['vin', 'odometer']].rename(columns={'odometer': 'odometer_date2'})


In [20]:
common_vins = set(df_date1['vin']).intersection(set(df_date2['vin']))
print(f"Number of common VINs after standardization: {len(common_vins)}")


Number of common VINs after standardization: 0


To ensure consistency in VIN formatting, the `vin` column was standardized by:
- Stripping any leading or trailing spaces.
- Converting all values to uppercase.

After applying this standardization, the dataset was re-filtered to recreate `df_date1` and `df_date2`, containing VINs and odometer readings for the two selected test dates. However, when checking for common VINs between these two dates, the result **remained zero**. This suggests that the absence of common VINs was not due to formatting inconsistencies.



In [21]:
# Get all unique test dates (remove the [:100] limitation)
unique_test_dates = df['test_date'].unique()

# Create a dictionary to store VINs for each test date
date_vin_dict = {date: set(df[df['test_date'] == date]['vin']) for date in unique_test_dates}

# Create a dictionary to store overlaps
overlap_dict = {}

# Compute overlaps
for i, date1 in enumerate(unique_test_dates):
    for date2 in unique_test_dates[i+1:]:
        common_vins = len(date_vin_dict[date1] & date_vin_dict[date2])
        if common_vins > 0:
            overlap_dict[(date1, date2)] = common_vins

# Display results
print(f"Number of date pairs with overlaps: {len(overlap_dict)}")
print("Sample overlaps:")
for (date1, date2), count in list(overlap_dict.items())[:5]:
    print(f"{date1} and {date2}: {count} common VINs")


Number of date pairs with overlaps: 447946
Sample overlaps:
2020-09-08T00:00:00.000000000 and 2024-08-26T00:00:00.000000000: 46 common VINs
2020-09-08T00:00:00.000000000 and 2021-08-10T00:00:00.000000000: 6 common VINs
2020-09-08T00:00:00.000000000 and 2023-06-29T00:00:00.000000000: 4 common VINs
2020-09-08T00:00:00.000000000 and 2020-11-09T00:00:00.000000000: 5 common VINs
2020-09-08T00:00:00.000000000 and 2020-11-07T00:00:00.000000000: 1 common VINs


To further investigate whether vehicles were tested on multiple dates, a broader approach was taken:
- All unique test dates in the dataset were identified.
- A dictionary was created to store VINs associated with each test date.
- Overlaps were computed between every pair of test dates by checking common VINs.

The results showed that there were **447,946 test date pairs with overlapping VINs**, indicating that many vehicles did undergo multiple tests, just not specifically on the two originally chosen test dates.

Some sample overlaps between test dates were displayed, showing instances where vehicles had common VINs across different dates. For example:
- **46 common VINs** between **2020-09-08 and 2024-08-26**.
- **6 common VINs** between **2020-09-08 and 2021-08-07**.
- Other test date pairs with small but significant overlaps.


In [22]:
# Selected test dates
date1 = '2020-09-08'
date2 = '2024-08-26'

# Filter data for the two selected test dates
df_date1 = df[df['test_date'] == date1][['vin', 'odometer']].rename(columns={'odometer': 'odometer_date1'})
df_date2 = df[df['test_date'] == date2][['vin', 'odometer']].rename(columns={'odometer': 'odometer_date2'})


In [23]:
# Merge the two DataFrames on 'vin'
df_combined = pd.merge(df_date1, df_date2, on='vin', how='inner')

# Display the resulting DataFrame
print(df_combined.head())


                 vin  odometer_date1  odometer_date2
0  1C4BJWEG7GL337305         39000.0         73000.0
1  1C4NJCBB2CD610286         70000.0        103000.0
2  1C4RJFCG0CC345855         65000.0         91000.0
3  1FA6P0HD3E5378424         93000.0        108000.0
4  1G11J5SX3EF225275         25000.0         33000.0


#### Key Observations:
- All vehicles showed an **increase in odometer readings**, confirming expected mileage accumulation.
- This comparison enables further analysis, such as:
  - **Estimating annual vehicle usage**
  - **Identifying anomalies or potential odometer rollbacks**
  - **Understanding vehicle longevity and wear patterns**


In [26]:
# Add columns for the selected test dates
df_combined['test_date1'] = date1
df_combined['test_date2'] = date2

# Display the resulting DataFrame
print(df_combined.head())


                 vin  odometer_date1  odometer_date2  mileage_difference  \
0  1C4BJWEG7GL337305         39000.0         73000.0             34000.0   
1  1C4NJCBB2CD610286         70000.0        103000.0             33000.0   
2  1C4RJFCG0CC345855         65000.0         91000.0             26000.0   
3  1FA6P0HD3E5378424         93000.0        108000.0             15000.0   
4  1G11J5SX3EF225275         25000.0         33000.0              8000.0   

   test_date1  test_date2  
0  2020-09-08  2024-08-26  
1  2020-09-08  2024-08-26  
2  2020-09-08  2024-08-26  
3  2020-09-08  2024-08-26  
4  2020-09-08  2024-08-26  


In [25]:
df_combined['mileage_difference'] = df_combined['odometer_date2'] - df_combined['odometer_date1']
print(df_combined[['vin', 'mileage_difference']].head())


                 vin  mileage_difference
0  1C4BJWEG7GL337305             34000.0
1  1C4NJCBB2CD610286             33000.0
2  1C4RJFCG0CC345855             26000.0
3  1FA6P0HD3E5378424             15000.0
4  1G11J5SX3EF225275              8000.0
